In [17]:
import requests
import time
from api_key import motion_api_key  # Ensure this imports the correct API key

class MotionAPIClient:
    def __init__(self, api_key, rate_limit=12, period=60):
        """
        Initializes the MotionAPIClient.

        Parameters:
            api_key (str): Your Motion API key.
            rate_limit (int): The number of allowed requests in the specified period.
            period (int): The period in seconds within which the rate limit applies.
        """
        self.api_key = api_key
        self.rate_limit = rate_limit
        self.period = period
        self.requests_made = 0
        self.start_time = time.time()

    def add_task_to_motion(self, task_name, task_description, due_date, priority, project_name, workspace_name):
        """
        Adds a task to Motion, checking and creating project and workspace if necessary.

        Parameters:
            task_name (str): The name of the task.
            task_description (str): The description of the task.
            due_date (str): The due date of the task in YYYY-MM-DD format.
            priority (str): The priority of the task (e.g., 'low', 'medium', 'high').
            project_name (str): The name of the project to which the task belongs.
            workspace_name (str): The name of the workspace to which the task belongs.

        Returns:
            dict: Response from the Motion API.
        """
        self._check_rate_limit()

        workspace_id = self._ensure_workspace_exists(workspace_name)
        project_id = self._ensure_project_exists(project_name, workspace_id)

        url = "https://api.usemotion.com/v1/tasks"
        headers = {
            "X-API-Key": self.api_key,
            "Content-Type": "application/json"
        }
        payload = {
            "name": task_name,
            "description": task_description,
            "due_date": due_date,
            "priority": priority,
            "project_id": project_id,
            "workspace_id": workspace_id
        }

        print(f"Adding task to URL: {url}")
        response = requests.post(url, headers=headers, json=payload)
        return self._handle_response(response, success_status_code=201, success_message="Task successfully added.")

    def _check_rate_limit(self):
        """
        Checks and enforces the rate limit.
        """
        current_time = time.time()
        elapsed_time = current_time - self.start_time

        if elapsed_time > self.period:
            self.requests_made = 0
            self.start_time = current_time

        if self.requests_made >= self.rate_limit:
            sleep_time = self.period - elapsed_time
            print(f"Rate limit reached. Sleeping for {sleep_time:.2f} seconds.")
            time.sleep(sleep_time)
            self.requests_made = 0
            self.start_time = time.time()

        self.requests_made += 1

    def _ensure_workspace_exists(self, workspace_name):
        """
        Ensures the workspace exists, creates it if it does not.

        Parameters:
            workspace_name (str): The name of the workspace.

        Returns:
            str: The ID of the workspace.
        """
        url = "https://api.usemotion.com/v1/workspaces"
        headers = {
            "X-API-Key": self.api_key,
            "Content-Type": "application/json"
        }

        print(f"Fetching workspaces from URL: {url}")
        #print(f"Using headers: {headers}")
        response = requests.get(url, headers=headers)
        workspaces = self._handle_response(response)

        for workspace in workspaces.get('workspaces', []):
            if workspace['name'] == workspace_name:
                return workspace['id']

        # Workspace does not exist, create it
        payload = {"name": workspace_name}
        print(f"Creating workspace at URL: {url}")
        response = requests.post(url, headers=headers, json=payload)
        workspace_data = self._handle_response(response, success_status_code=201, success_message=f"Workspace '{workspace_name}' created.")
        return workspace_data['id']

    def _ensure_project_exists(self, project_name, workspace_id):
        """
        Ensures the project exists, creates it if it does not.

        Parameters:
            project_name (str): The name of the project.
            workspace_id (str): The ID of the workspace.

        Returns:
            str: The ID of the project.
        """
        url = f"https://api.usemotion.com/v1/projects?workspaceId={workspace_id}"
        headers = {
            "X-API-Key": self.api_key,
            "Content-Type": "application/json"
        }

        print(f"Fetching projects from URL: {url}")
        response = requests.get(url, headers=headers)
        projects = self._handle_response(response)

        for project in projects.get('projects', []):
            if project['name'] == project_name:
                return project['id']

        # Project does not exist, create it
        payload = {"name": project_name, "workspaceId": workspace_id}
        print(f"Creating project at URL: {url}")
        response = requests.post(url, headers=headers, json=payload)
        project_data = self._handle_response(response, success_status_code=201, success_message=f"Project '{project_name}' created.")
        return project_data['id']

    def _handle_response(self, response, success_status_code=200, success_message=None):
        """
        Handles the API response.

        Parameters:
            response (requests.Response): The response object.
            success_status_code (int): The expected status code for a successful response.
            success_message (str): The message to print if the request is successful.

        Returns:
            dict: The JSON content of the response.

        Raises:
            Exception: If the response status code is not as expected or if JSON decoding fails.
        """
        try:
            response.raise_for_status()
            if response.status_code == success_status_code and success_message:
                print(success_message)

            if 'application/json' in response.headers.get('Content-Type', ''):
                return response.json()
            else:
                print(f"Unexpected content type: {response.headers.get('Content-Type')}")
                return response.text
        except requests.exceptions.HTTPError as http_err:
            if response.status_code == 401:
                print("Unauthorized access - please check your API key and permissions.")
            print(f"HTTP error occurred: {http_err}")
            print(f"Response content: {response.content}")
            raise
        except requests.exceptions.RequestException as req_err:
            print(f"Request error occurred: {req_err}")
            raise
        except ValueError as json_err:
            print(f"JSON decode error: {json_err}")
            print(f"Response content: {response.content}")
            raise

# Example usage:
client = MotionAPIClient(motion_api_key)

task_name = "Complete Project Proposal"
task_description = "Finish the project proposal for the new client by the end of the week."
due_date = "2024-07-10"
priority = "high"
project_name = "New Client Project"
workspace_name = "Client Workspaces"

response = client.add_task_to_motion(task_name, task_description, due_date, priority, project_name, workspace_name)
print(response)


Fetching workspaces from URL: https://api.usemotion.com/v1/workspaces
Unauthorized access - please check your API key and permissions.
HTTP error occurred: 401 Client Error: Unauthorized for url: https://api.usemotion.com/v1/workspaces
Response content: b'{"message":"Unauthorized","statusCode":401}'


HTTPError: 401 Client Error: Unauthorized for url: https://api.usemotion.com/v1/workspaces